In [1]:
import pandas as pd
import requests, re
from bs4 import BeautifulSoup

In [2]:
#remove comments because it confuses parser
comm = re.compile("<!--|-->")
def make_soup(url):
    page = requests.get(url).text
    soup = BeautifulSoup(comm.sub('', page), 'html.parser')
    return soup

In [3]:
def get_college_data(url):
    data = {}
    soup = make_soup(url)

    passing = soup.find('table', id='passing')
    last_row = passing.tbody.find_all('tr')[-1]
    data['CONF'] = last_row.find_all('td')[1].get_text()
    tds = passing.tfoot.find_all('td')
    data['COLL_CMP'] = tds[5].get_text()
    data['COLL_ATT'] = tds[6].get_text()
    data['COLL_YDS'] = tds[8].get_text()
    data['COLL_TD'] = tds[11].get_text()
    data['COLL_INT'] = tds[12].get_text()

    rushing = soup.find('table', id='rushing').tfoot
    tds = rushing.find_all('td')
    data['COLL_RUSH_ATT'] = tds[5].get_text()
    data['COLL_RUSH_YDS'] = tds[6].get_text()
    data['COLL_RUSH_TDS'] = tds[8].get_text()

    return data

In [19]:
QB_NAMES = {'Justin Herbert', 'Dwayne Haskins', 'Daniel Jones', 'Will Grier', 'Drew Lock', 'Ryan Finley', 'Gardner Minshew', 'Clayton Thorson', 'Jarrett Stidham', 'Deondre Francois', 'Shea Patterson', 'Nathan Stanley', 'Kyle Shurmur', 'Kyler Murray'}
COLUMN_NAMES =["PLAYER", "COLLEGE", "CONF", "COLL_CMP", "COLL_ATT", "COLL_YDS", "COLL_TD", "COLL_INT", "COLL_RUSH_ATT", "COLL_RUSH_YDS", "COLL_RUSH_TDS"]

BASE_URL = 'https://www.sports-reference.com/cfb/years/2018-passing.html'
page = requests.get(BASE_URL).text
soup = BeautifulSoup(page, 'html.parser')

table = soup.find(id='passing')


In [20]:
data = []

for tr in table.tbody.find_all('tr', class_=lambda x: x != 'thead'):
    player = {}
    tds = tr.find_all('td')
    name = tds[0].get_text()
    if name in QB_NAMES:
        player['PLAYER'] = name
        player['COLLEGE'] = tds[1].get_text()
        player['CONF'] = tds[2].get_text()
        player.update(get_college_data('https://www.sports-reference.com' + tds[0].a['href']))
        data.append(player)
df = pd.DataFrame(data, columns=COLUMN_NAMES)
df

,PLAYER,COLLEGE,CONF,COLL_CMP,COLL_ATT,COLL_YDS,COLL_TD,COLL_INT,COLL_RUSH_ATT,COLL_RUSH_YDS,COLL_RUSH_TDS
0,Kyler Murray,Oklahoma,Big 12,331,482,5098,48,14,190,1369,12
1,Will Grier,West Virginia,Big 12,621,945,8556,81,23,147,148,7
2,Dwayne Haskins,Ohio State,Big Ten,388,553,5145,51,9,97,208,4
3,Ryan Finley,North Carolina State,ACC,919,1429,11008,62,28,209,386,5
4,Shea Patterson,Michigan,Big Ten,426,682,5503,44,17,154,421,3
5,Justin Herbert,Oregon,Pac-12,522,832,6904,62,17,169,517,9
6,Drew Lock,Missouri,SEC,860,1515,11820,96,39,196,407,9
7,Kyle Shurmur,Vanderbilt,SEC,704,1227,8579,63,29,133,-340,3
8,Gardner Minshew,Washington State,Pac-12,726,1119,7964,60,20,90,27,3
9,Nathan Stanley,Iowa,Big Ten,415,726,5132,49,15,81,-93,1


In [21]:
df.to_csv('data/test_data.csv')